Using Keras, SELU multilayer perceptrons and Hyperopt on Tox21
====


In [1]:
import numpy as np
import pandas as pd
from scipy import io
    
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers.noise import AlphaDropout
from keras import optimizers
from keras import regularizers
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import backend as K

from hyperopt import fmin, tpe, hp, STATUS_OK, STATUS_FAIL, Trials

from sklearn import preprocessing
from sklearn.metrics import roc_auc_score
import sklearn.utils 

import tempfile

import tensorflow as tf

source_directory = "/media/matthias/Big-disk/tox21-hochreiter/"

# load dense features and labels
y_tr = pd.read_csv(source_directory + 'tox21_labels_train.csv.gz', index_col=0, compression="gzip")
y_te = pd.read_csv(source_directory + 'tox21_labels_test.csv.gz', index_col=0, compression="gzip")
x_tr_dense = pd.read_csv(source_directory + 'tox21_dense_train.csv.gz', index_col=0, compression="gzip").values
x_te_dense = pd.read_csv(source_directory + 'tox21_dense_test.csv.gz', index_col=0, compression="gzip").values

# load sparse features
x_tr_sparse = io.mmread(source_directory + 'tox21_sparse_train.mtx.gz').tocsc()
x_te_sparse = io.mmread(source_directory + 'tox21_sparse_test.mtx.gz').tocsc()

# filter out very sparse features. combine filtered sparse features with dense features
sparse_col_idx = ((x_tr_sparse > 0).mean(0) > 0.05).A.ravel()
x_tr = np.hstack([x_tr_dense, x_tr_sparse[:, sparse_col_idx].A])
x_te = np.hstack([x_te_dense, x_te_sparse[:, sparse_col_idx].A])

# scale input features to zero mean and unit variance (might be important for SELU network)
scaler = preprocessing.StandardScaler().fit(x_tr)
x_tr = scaler.transform(x_tr)
x_te = scaler.transform(x_te)



Using TensorFlow backend.


In [2]:
# All target assays: ['NR.AhR', 'NR.AR', 'NR.AR.LBD', 'NR.Aromatase', 'NR.ER', 'NR.ER.LBD',
#       'NR.PPAR.gamma', 'SR.ARE', 'SR.ATAD5', 'SR.HSE', 'SR.MMP', 'SR.p53']

def data(target):
    """
    Returns data for a specific assays (i.e., samples where result for assay is available)
    """
    
    # filter out data where no results for target assay are available    
    rows_tr = np.isfinite(y_tr[target]).values
    rows_te = np.isfinite(y_te[target]).values
    
    x_train = x_tr[rows_tr] 
    y_train = y_tr[target][rows_tr]
    x_test = x_te[rows_te]
    y_test = y_te[target][rows_te]   
    
    return x_train, y_train, x_test, y_test


In [10]:
def f_nn(p):
    # add constant parameters to parameter dictionary (workaround for hyperopt, take care to set global variable!)
    p = {**p, **constant_parameters}
    
    print("\n")
    print(p)
    
    try:
        max_epochs = 1000

        # some parameters need to be cast to integer
        p['hidden layers'] = int(p['hidden layers'])
        p['hidden units'] = int(p['hidden units'])
        p['batch size'] = int(p['batch size'])

        number_of_features = x_train.shape[1]
        
        # clear session (to avoid clutter from old models)
        K.clear_session()
        
        # the model is a classical multilayer perceptron with SELU activations and Alpha dropout
        model = Sequential()
        model.add(Dense(p['hidden units'], input_dim=number_of_features, 
                        activation='selu', kernel_initializer=p['kernel initializer']))
        for i in range(p['hidden layers']):
            if p['shape'] == 'pyramid':
                # pyramid shape: each layer has half the units of the previous layer (but at least 2 units)
                number_of_units_in_this_layer = max(int(p['hidden units'] / pow(2, i+1)), 2) 
            else:
                number_of_units_in_this_layer = p['hidden units']
            model.add(AlphaDropout(p['dropout rate']))
            model.add(Dense(p['hidden units'], activation='selu', kernel_initializer=p['kernel initializer']))
        model.add(Dense(1, activation='sigmoid'))

        if p['optimizer'] == 'sgd':
            optimizer=optimizers.sgd(lr=p['learning rate'])
        elif p['optimizer'] == 'adam':
            optimizer=optimizers.adam(lr=p['learning rate'])
        elif p['optimizer'] == 'RMSprop':
            optimizer=optimizers.RMSprop(lr=p['learning rate'])
        elif p['optimizer'] == 'Adadelta':
            optimizer=optimizers.RMSprop(lr=p['learning rate'])


        model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['binary_accuracy'])

        # weigh classes (in case labels are unbalanced)
        class_weight = sklearn.utils.class_weight.compute_class_weight('balanced', np.unique(y_train), y_train)

        print('Class weights:')
        print(class_weight)
        _, tmpfn = tempfile.mkstemp()

        callbacks = [EarlyStopping(patience=p['patience']), ModelCheckpoint(tmpfn, save_best_only=True, save_weights_only=True)]
        model.fit(x_train, y_train, epochs=max_epochs, batch_size=p['batch size'], validation_data=(x_test, y_test), 
              callbacks=callbacks, class_weight=class_weight, verbose=0)

        # load model weights of epoch that had best results
        model.load_weights(tmpfn)

        p_te = model.predict_proba(x_test)

        # add column for other class, adding up to probability of 1
        new_col = np.subtract(1,p_te).reshape((p_te.shape[0],1))
        p_te_both_classes = np.append(new_col,p_te, 1)

        auc_te = roc_auc_score(y_test, p_te_both_classes[:, 1])
        print("%15s score on test: %3.5f" % (target, auc_te))

        return {'loss': -auc_te, 'status': STATUS_OK, 'model': model}
    
    except tf.errors.ResourceExhaustedError: 
        # fail gracefully if model is too large to fit into memory (so hyperparameter search can continue)
        print("ResourceExhaustedError occurred, skipping...")
        return {'status': STATUS_FAIL}

In [3]:
space = {
    'shape': hp.choice('shape', ['square', 'pyramid']),
    'hidden layers': hp.quniform('hidden layers', 0, 10, 1),
    'hidden units': hp.quniform('hidden units', 200, 2048, 1),
    'dropout rate': hp.uniform('dropout rate', .001, .5),
    'batch size' : hp.quniform('batch size', 10, 100, 1),
    'learning rate': hp.uniform('learning rate', 1e-06, 1e-04 ),
    'optimizer': hp.choice('optimizer',['sgd','adam']),
    'kernel initializer': hp.choice('kernel initializer',['lecun_normal','he_normal', 'glorot_uniform']),
    'l2 kernel regularization': hp.uniform('l2 kernel regularization', 1e-06, 1e-05)}    
    
trials = Trials()
best = fmin(f_nn, space, algo=tpe.suggest, max_evals=50, trials=trials)

print('best: ')
print(best)



{'batch size': 34.0, 'dropout rate': 0.07970398031653203, 'hidden layers': 6.0, 'hidden units': 1439.0, 'kernel initializer': 'he_normal', 'l2 kernel regularization': 4.570848329984473e-06, 'learning rate': 1.7760650984633667e-05, 'optimizer': 'adam', 'shape': 'pyramid'}
 32/610 [>.............................] - ETA: 0s         NR.AhR score on test: 0.82197


{'batch size': 20.0, 'dropout rate': 0.07149632323164115, 'hidden layers': 9.0, 'hidden units': 1999.0, 'kernel initializer': 'glorot_uniform', 'l2 kernel regularization': 5.341642875118799e-06, 'learning rate': 7.128224107068452e-05, 'optimizer': 'sgd', 'shape': 'square'}
 32/610 [>.............................] - ETA: 0s         NR.AhR score on test: 0.78562


{'batch size': 65.0, 'dropout rate': 0.4171895094613189, 'hidden layers': 8.0, 'hidden units': 1057.0, 'kernel initializer': 'lecun_normal', 'l2 kernel regularization': 1.7659788690358359e-06, 'learning rate': 5.815697560092928e-05, 'optimizer': 'sgd', 'shape': 'pyramid

 32/610 [>.............................] - ETA: 0s         NR.AhR score on test: 0.81008


{'batch size': 43.0, 'dropout rate': 0.24371606269248938, 'hidden layers': 3.0, 'hidden units': 783.0, 'kernel initializer': 'he_normal', 'l2 kernel regularization': 3.561382036885765e-06, 'learning rate': 2.0922619527506757e-05, 'optimizer': 'adam', 'shape': 'pyramid'}
 32/610 [>.............................] - ETA: 0s         NR.AhR score on test: 0.79720


{'batch size': 52.0, 'dropout rate': 0.02455922161449304, 'hidden layers': 5.0, 'hidden units': 1325.0, 'kernel initializer': 'glorot_uniform', 'l2 kernel regularization': 6.076351534048634e-06, 'learning rate': 3.3000701278864124e-05, 'optimizer': 'adam', 'shape': 'pyramid'}
 32/610 [>.............................] - ETA: 0s         NR.AhR score on test: 0.46741


{'batch size': 53.0, 'dropout rate': 0.2806510239225377, 'hidden layers': 3.0, 'hidden units': 881.0, 'kernel initializer': 'he_normal', 'l2 kernel regularization': 4.234249370626

 32/610 [>.............................] - ETA: 1s         NR.AhR score on test: 0.56843


{'batch size': 48.0, 'dropout rate': 0.4519645165295243, 'hidden layers': 7.0, 'hidden units': 252.0, 'kernel initializer': 'glorot_uniform', 'l2 kernel regularization': 8.335776641085155e-06, 'learning rate': 6.862756000386326e-05, 'optimizer': 'adam', 'shape': 'square'}
 32/610 [>.............................] - ETA: 1s         NR.AhR score on test: 0.80679


{'batch size': 62.0, 'dropout rate': 0.07647742589205056, 'hidden layers': 9.0, 'hidden units': 988.0, 'kernel initializer': 'lecun_normal', 'l2 kernel regularization': 5.794125792648407e-06, 'learning rate': 4.597745832972281e-05, 'optimizer': 'sgd', 'shape': 'square'}
 32/610 [>.............................] - ETA: 0s         NR.AhR score on test: 0.77567


{'batch size': 29.0, 'dropout rate': 0.3424094236187401, 'hidden layers': 6.0, 'hidden units': 574.0, 'kernel initializer': 'glorot_uniform', 'l2 kernel regularization': 8.01652509194

Train and evaluate model with best hyperparameters
----------

In [10]:
from hyperopt import space_eval 
training_result = f_nn(space_eval(space, best))



{'batch size': 35.0, 'dropout rate': 0.029867293022818375, 'hidden layers': 10.0, 'hidden units': 419.0, 'kernel initializer': 'glorot_uniform', 'l2 kernel regularization': 7.0412947425240335e-06, 'learning rate': 5.5633772459955735e-05, 'optimizer': 'adam', 'shape': 'square'}
 32/610 [>.............................] - ETA: 1s         NR.AhR score on test: 0.83457


Calculate ROC AUC on *training* data to see if we are over- or under-fitting
-----

In [15]:
model = training_result['model']
for target in ['NR.AhR']:    
    p_tr = model.predict_proba(x_train)
    # add column for other class, adding up to probability of 1
    new_col = np.subtract(1,p_tr).reshape((p_tr.shape[0],1))
    p_tr_both_classes = np.append(new_col,p_tr, 1)
    
    auc_tr = roc_auc_score(y_train, p_tr_both_classes[:, 1])
    print("%15s score on training: %3.5f" % (target, auc_tr))

7680/8441 [==========================>...] - ETA: 0s         NR.AhR score on training: 0.81494


Seems like we are not overfitting

In [4]:
target = 'NR.AhR'
x_train, y_train, x_test, y_test = data(target)

space = {
    'batch size' : hp.quniform('batch size', 10, 100, 1),
    'hidden layers': hp.quniform('hidden layers', 0, 16, 1),
    'hidden units': hp.quniform('hidden units', 100, 1000, 1),
    'dropout rate': hp.uniform('dropout rate', .01, .1),
    'learning rate': hp.uniform('learning rate', 1e-07, 1e-05 )}

constant_parameters = {
    'shape': 'square',
    'optimizer': 'sgd',
    'kernel initializer': 'lecun_normal',
    'patience': 30
}

trials = Trials()
best = fmin(f_nn, space, algo=tpe.suggest, max_evals=100, trials=trials)

print('best: ')
print(best)





{'batch size': 83.0, 'dropout rate': 0.05122583014934502, 'hidden layers': 9.0, 'hidden units': 406.0, 'learning rate': 7.130664298367703e-07, 'shape': 'square', 'optimizer': 'sgd', 'kernel initializer': 'lecun_normal', 'patience': 30}
Class weights:
[ 0.56575067  4.30224261]
Train on 8441 samples, validate on 610 samples
Epoch 1/1000
1s - loss: 0.7559 - binary_accuracy: 0.5613 - val_loss: 0.5905 - val_binary_accuracy: 0.6918
Epoch 2/1000
0s - loss: 0.7615 - binary_accuracy: 0.5566 - val_loss: 0.5846 - val_binary_accuracy: 0.7016
Epoch 3/1000
0s - loss: 0.7520 - binary_accuracy: 0.5624 - val_loss: 0.5788 - val_binary_accuracy: 0.7066
Epoch 4/1000
0s - loss: 0.7547 - binary_accuracy: 0.5592 - val_loss: 0.5732 - val_binary_accuracy: 0.7098
Epoch 5/1000
0s - loss: 0.7510 - binary_accuracy: 0.5745 - val_loss: 0.5679 - val_binary_accuracy: 0.7197
Epoch 6/1000
0s - loss: 0.7462 - binary_accuracy: 0.5638 - val_loss: 0.5627 - val_binary_accuracy: 0.7197
Epoch 7/1000
0s - loss: 0.7508 - binar

0s - loss: 0.5973 - binary_accuracy: 0.6940 - val_loss: 0.4394 - val_binary_accuracy: 0.8492
Epoch 76/1000
0s - loss: 0.6047 - binary_accuracy: 0.6928 - val_loss: 0.4394 - val_binary_accuracy: 0.8525
Epoch 77/1000
0s - loss: 0.5968 - binary_accuracy: 0.6978 - val_loss: 0.4394 - val_binary_accuracy: 0.8525
Epoch 78/1000
0s - loss: 0.6017 - binary_accuracy: 0.6942 - val_loss: 0.4395 - val_binary_accuracy: 0.8525
Epoch 79/1000
0s - loss: 0.5982 - binary_accuracy: 0.6979 - val_loss: 0.4395 - val_binary_accuracy: 0.8557
Epoch 80/1000
0s - loss: 0.5957 - binary_accuracy: 0.6966 - val_loss: 0.4396 - val_binary_accuracy: 0.8557
Epoch 81/1000
0s - loss: 0.6001 - binary_accuracy: 0.6920 - val_loss: 0.4397 - val_binary_accuracy: 0.8557
Epoch 82/1000
0s - loss: 0.5916 - binary_accuracy: 0.7015 - val_loss: 0.4398 - val_binary_accuracy: 0.8557
Epoch 83/1000
0s - loss: 0.5905 - binary_accuracy: 0.6994 - val_loss: 0.4400 - val_binary_accuracy: 0.8557
Epoch 84/1000
0s - loss: 0.5941 - binary_accuracy: 

1s - loss: 0.3620 - binary_accuracy: 0.8763 - val_loss: 0.3876 - val_binary_accuracy: 0.8754
Epoch 42/1000
1s - loss: 0.3608 - binary_accuracy: 0.8776 - val_loss: 0.3854 - val_binary_accuracy: 0.8754
Epoch 43/1000
1s - loss: 0.3581 - binary_accuracy: 0.8766 - val_loss: 0.3839 - val_binary_accuracy: 0.8754
Epoch 44/1000
1s - loss: 0.3658 - binary_accuracy: 0.8762 - val_loss: 0.3822 - val_binary_accuracy: 0.8754
Epoch 45/1000
1s - loss: 0.3580 - binary_accuracy: 0.8768 - val_loss: 0.3799 - val_binary_accuracy: 0.8754
Epoch 46/1000
1s - loss: 0.3566 - binary_accuracy: 0.8774 - val_loss: 0.3780 - val_binary_accuracy: 0.8754
Epoch 47/1000
1s - loss: 0.3556 - binary_accuracy: 0.8787 - val_loss: 0.3760 - val_binary_accuracy: 0.8754
Epoch 48/1000
1s - loss: 0.3493 - binary_accuracy: 0.8782 - val_loss: 0.3746 - val_binary_accuracy: 0.8754
Epoch 49/1000
1s - loss: 0.3610 - binary_accuracy: 0.8764 - val_loss: 0.3705 - val_binary_accuracy: 0.8754
Epoch 50/1000
1s - loss: 0.3534 - binary_accuracy: 

1s - loss: 0.3212 - binary_accuracy: 0.8775 - val_loss: 0.3033 - val_binary_accuracy: 0.8770
Epoch 119/1000
1s - loss: 0.3231 - binary_accuracy: 0.8795 - val_loss: 0.3032 - val_binary_accuracy: 0.8770
Epoch 120/1000
1s - loss: 0.3208 - binary_accuracy: 0.8790 - val_loss: 0.3026 - val_binary_accuracy: 0.8770
Epoch 121/1000
1s - loss: 0.3188 - binary_accuracy: 0.8794 - val_loss: 0.3026 - val_binary_accuracy: 0.8770
Epoch 122/1000
1s - loss: 0.3219 - binary_accuracy: 0.8786 - val_loss: 0.3021 - val_binary_accuracy: 0.8770
Epoch 123/1000
1s - loss: 0.3235 - binary_accuracy: 0.8780 - val_loss: 0.3012 - val_binary_accuracy: 0.8803
Epoch 124/1000
1s - loss: 0.3203 - binary_accuracy: 0.8818 - val_loss: 0.3008 - val_binary_accuracy: 0.8803
Epoch 125/1000
1s - loss: 0.3147 - binary_accuracy: 0.8805 - val_loss: 0.3008 - val_binary_accuracy: 0.8803
Epoch 126/1000
1s - loss: 0.3177 - binary_accuracy: 0.8807 - val_loss: 0.3007 - val_binary_accuracy: 0.8803
Epoch 127/1000
1s - loss: 0.3163 - binary_a

1s - loss: 0.3069 - binary_accuracy: 0.8816 - val_loss: 0.2812 - val_binary_accuracy: 0.8951
Epoch 195/1000
1s - loss: 0.3073 - binary_accuracy: 0.8808 - val_loss: 0.2810 - val_binary_accuracy: 0.8951
Epoch 196/1000
1s - loss: 0.3055 - binary_accuracy: 0.8825 - val_loss: 0.2811 - val_binary_accuracy: 0.8951
Epoch 197/1000
0s - loss: 0.3039 - binary_accuracy: 0.8822 - val_loss: 0.2811 - val_binary_accuracy: 0.8951
Epoch 198/1000
1s - loss: 0.3079 - binary_accuracy: 0.8860 - val_loss: 0.2807 - val_binary_accuracy: 0.8951
Epoch 199/1000
1s - loss: 0.3051 - binary_accuracy: 0.8828 - val_loss: 0.2807 - val_binary_accuracy: 0.8951
Epoch 200/1000
1s - loss: 0.2979 - binary_accuracy: 0.8852 - val_loss: 0.2801 - val_binary_accuracy: 0.8984
Epoch 201/1000
1s - loss: 0.3012 - binary_accuracy: 0.8840 - val_loss: 0.2800 - val_binary_accuracy: 0.8984
Epoch 202/1000
1s - loss: 0.3017 - binary_accuracy: 0.8845 - val_loss: 0.2798 - val_binary_accuracy: 0.8984
Epoch 203/1000
1s - loss: 0.3046 - binary_a

...

0s - loss: 0.2609 - binary_accuracy: 0.8978 - val_loss: 0.2577 - val_binary_accuracy: 0.8869
Epoch 889/1000
0s - loss: 0.2582 - binary_accuracy: 0.8968 - val_loss: 0.2577 - val_binary_accuracy: 0.8869
Epoch 890/1000
0s - loss: 0.2611 - binary_accuracy: 0.8959 - val_loss: 0.2576 - val_binary_accuracy: 0.8869
Epoch 891/1000
0s - loss: 0.2627 - binary_accuracy: 0.8934 - val_loss: 0.2576 - val_binary_accuracy: 0.8869
Epoch 892/1000
0s - loss: 0.2566 - binary_accuracy: 0.8944 - val_loss: 0.2576 - val_binary_accuracy: 0.8869
Epoch 893/1000
0s - loss: 0.2636 - binary_accuracy: 0.8950 - val_loss: 0.2576 - val_binary_accuracy: 0.8869
Epoch 894/1000
0s - loss: 0.2622 - binary_accuracy: 0.8941 - val_loss: 0.2576 - val_binary_accuracy: 0.8869
Epoch 895/1000
0s - loss: 0.2605 - binary_accuracy: 0.8953 - val_loss: 0.2576 - val_binary_accuracy: 0.8869
Epoch 896/1000
0s - loss: 0.2578 - binary_accuracy: 0.8925 - val_loss: 0.2576 - val_binary_accuracy: 0.8869
Epoch 897/1000
0s - loss: 0.2603 - binary_a

0s - loss: 0.2599 - binary_accuracy: 0.8952 - val_loss: 0.2573 - val_binary_accuracy: 0.8885
Epoch 965/1000
0s - loss: 0.2622 - binary_accuracy: 0.8946 - val_loss: 0.2573 - val_binary_accuracy: 0.8885
Epoch 966/1000
0s - loss: 0.2590 - binary_accuracy: 0.8936 - val_loss: 0.2573 - val_binary_accuracy: 0.8885
Epoch 967/1000
0s - loss: 0.2581 - binary_accuracy: 0.8976 - val_loss: 0.2572 - val_binary_accuracy: 0.8885
Epoch 968/1000
0s - loss: 0.2586 - binary_accuracy: 0.8997 - val_loss: 0.2572 - val_binary_accuracy: 0.8885
Epoch 969/1000
0s - loss: 0.2588 - binary_accuracy: 0.8948 - val_loss: 0.2572 - val_binary_accuracy: 0.8885
Epoch 970/1000
0s - loss: 0.2549 - binary_accuracy: 0.9012 - val_loss: 0.2572 - val_binary_accuracy: 0.8885
Epoch 971/1000
0s - loss: 0.2619 - binary_accuracy: 0.8957 - val_loss: 0.2572 - val_binary_accuracy: 0.8885
Epoch 972/1000
0s - loss: 0.2561 - binary_accuracy: 0.8989 - val_loss: 0.2572 - val_binary_accuracy: 0.8885
Epoch 973/1000
0s - loss: 0.2572 - binary_a

0s - loss: 0.7988 - binary_accuracy: 0.5133 - val_loss: 0.8020 - val_binary_accuracy: 0.3443
Epoch 38/1000
0s - loss: 0.7885 - binary_accuracy: 0.5195 - val_loss: 0.8016 - val_binary_accuracy: 0.3459
Epoch 39/1000
0s - loss: 0.7941 - binary_accuracy: 0.5106 - val_loss: 0.8013 - val_binary_accuracy: 0.3459
Epoch 40/1000
0s - loss: 0.8014 - binary_accuracy: 0.5055 - val_loss: 0.8009 - val_binary_accuracy: 0.3459
Epoch 41/1000
0s - loss: 0.7977 - binary_accuracy: 0.5116 - val_loss: 0.8006 - val_binary_accuracy: 0.3459
Epoch 42/1000
0s - loss: 0.7988 - binary_accuracy: 0.5132 - val_loss: 0.8002 - val_binary_accuracy: 0.3459
Epoch 43/1000
0s - loss: 0.7950 - binary_accuracy: 0.5114 - val_loss: 0.7999 - val_binary_accuracy: 0.3459
Epoch 44/1000
0s - loss: 0.7892 - binary_accuracy: 0.5175 - val_loss: 0.7995 - val_binary_accuracy: 0.3475
Epoch 45/1000
0s - loss: 0.8046 - binary_accuracy: 0.5010 - val_loss: 0.7992 - val_binary_accuracy: 0.3475
Epoch 46/1000
0s - loss: 0.7904 - binary_accuracy: 

0s - loss: 0.7911 - binary_accuracy: 0.5123 - val_loss: 0.7758 - val_binary_accuracy: 0.3738
Epoch 115/1000
0s - loss: 0.7953 - binary_accuracy: 0.5152 - val_loss: 0.7754 - val_binary_accuracy: 0.3738
Epoch 116/1000
0s - loss: 0.7839 - binary_accuracy: 0.5184 - val_loss: 0.7751 - val_binary_accuracy: 0.3738
Epoch 117/1000
0s - loss: 0.7802 - binary_accuracy: 0.5271 - val_loss: 0.7748 - val_binary_accuracy: 0.3738
Epoch 118/1000
0s - loss: 0.7920 - binary_accuracy: 0.5172 - val_loss: 0.7744 - val_binary_accuracy: 0.3738
Epoch 119/1000
0s - loss: 0.7907 - binary_accuracy: 0.5152 - val_loss: 0.7741 - val_binary_accuracy: 0.3738
Epoch 120/1000
0s - loss: 0.7813 - binary_accuracy: 0.5281 - val_loss: 0.7738 - val_binary_accuracy: 0.3738
Epoch 121/1000
0s - loss: 0.7870 - binary_accuracy: 0.5176 - val_loss: 0.7734 - val_binary_accuracy: 0.3738
Epoch 122/1000
0s - loss: 0.7738 - binary_accuracy: 0.5287 - val_loss: 0.7731 - val_binary_accuracy: 0.3738
Epoch 123/1000
0s - loss: 0.7906 - binary_a

KeyboardInterrupt: 

In [5]:
space = {
    'batch size' : hp.quniform('batch size', 10, 100, 1),
    'hidden layers': hp.quniform('hidden layers', 0, 16, 1),
    'hidden units': hp.quniform('hidden units', 100, 1000, 1),
    'dropout rate': hp.uniform('dropout rate', .01, .1),
    'learning rate': hp.uniform('learning rate', 1e-07, 1e-05 )}

constant_parameters = {
    'shape': 'square',
    'optimizer': 'adam',
    'kernel initializer': 'lecun_normal',
    'patience': 30
}

trials = Trials()
best = fmin(f_nn, space, algo=tpe.suggest, max_evals=100, trials=trials)

print('best: ')
print(best)




{'batch size': 29.0, 'dropout rate': 0.08208304547004898, 'hidden layers': 5.0, 'hidden units': 654.0, 'learning rate': 7.075988301780786e-06, 'shape': 'square', 'optimizer': 'adam', 'kernel initializer': 'lecun_normal', 'patience': 30}
Class weights:
[ 0.56575067  4.30224261]
Train on 8441 samples, validate on 610 samples
Epoch 1/1000
2s - loss: 0.3590 - binary_accuracy: 0.8652 - val_loss: 0.3706 - val_binary_accuracy: 0.8869
Epoch 2/1000
2s - loss: 0.3041 - binary_accuracy: 0.8825 - val_loss: 0.3282 - val_binary_accuracy: 0.8951
Epoch 3/1000
2s - loss: 0.2851 - binary_accuracy: 0.8891 - val_loss: 0.3138 - val_binary_accuracy: 0.8918
Epoch 4/1000
2s - loss: 0.2707 - binary_accuracy: 0.8931 - val_loss: 0.3089 - val_binary_accuracy: 0.8902
Epoch 5/1000
1s - loss: 0.2614 - binary_accuracy: 0.8933 - val_loss: 0.2986 - val_binary_accuracy: 0.8836
Epoch 6/1000
1s - loss: 0.2616 - binary_accuracy: 0.8974 - val_loss: 0.2953 - val_binary_accuracy: 0.8934
Epoch 7/1000
1s - loss: 0.2507 - bina

0s - loss: 0.2370 - binary_accuracy: 0.9068 - val_loss: 0.2481 - val_binary_accuracy: 0.8967
Epoch 13/1000
0s - loss: 0.2358 - binary_accuracy: 0.9069 - val_loss: 0.2531 - val_binary_accuracy: 0.9016
Epoch 14/1000
0s - loss: 0.2374 - binary_accuracy: 0.9039 - val_loss: 0.2464 - val_binary_accuracy: 0.8934
Epoch 15/1000
0s - loss: 0.2309 - binary_accuracy: 0.9078 - val_loss: 0.2482 - val_binary_accuracy: 0.8951
Epoch 16/1000
0s - loss: 0.2326 - binary_accuracy: 0.9082 - val_loss: 0.2438 - val_binary_accuracy: 0.8951
Epoch 17/1000
0s - loss: 0.2288 - binary_accuracy: 0.9119 - val_loss: 0.2444 - val_binary_accuracy: 0.8951
Epoch 18/1000
0s - loss: 0.2292 - binary_accuracy: 0.9109 - val_loss: 0.2490 - val_binary_accuracy: 0.9000
Epoch 19/1000
0s - loss: 0.2264 - binary_accuracy: 0.9107 - val_loss: 0.2479 - val_binary_accuracy: 0.9000
Epoch 20/1000
0s - loss: 0.2234 - binary_accuracy: 0.9113 - val_loss: 0.2497 - val_binary_accuracy: 0.8967
Epoch 21/1000
0s - loss: 0.2233 - binary_accuracy: 

3s - loss: 0.1551 - binary_accuracy: 0.9382 - val_loss: 0.3096 - val_binary_accuracy: 0.8803
Epoch 39/1000
3s - loss: 0.1558 - binary_accuracy: 0.9405 - val_loss: 0.3412 - val_binary_accuracy: 0.8672
Epoch 40/1000
3s - loss: 0.1531 - binary_accuracy: 0.9412 - val_loss: 0.2945 - val_binary_accuracy: 0.8918
Epoch 41/1000
3s - loss: 0.1497 - binary_accuracy: 0.9415 - val_loss: 0.3062 - val_binary_accuracy: 0.8918
Epoch 42/1000
3s - loss: 0.1459 - binary_accuracy: 0.9448 - val_loss: 0.3536 - val_binary_accuracy: 0.8738
Epoch 43/1000
3s - loss: 0.1478 - binary_accuracy: 0.9427 - val_loss: 0.2862 - val_binary_accuracy: 0.8934
Epoch 44/1000
3s - loss: 0.1445 - binary_accuracy: 0.9444 - val_loss: 0.3086 - val_binary_accuracy: 0.8934
Epoch 45/1000
3s - loss: 0.1468 - binary_accuracy: 0.9428 - val_loss: 0.2957 - val_binary_accuracy: 0.9082
 32/610 [>.............................] - ETA: 0s         NR.AhR score on test: 0.89245


{'batch size': 34.0, 'dropout rate': 0.014983864767755647, 'hidden 

KeyboardInterrupt: 

In [9]:
space = {
    'batch size' : hp.quniform('batch size', 10, 100, 1),
    'hidden layers': hp.quniform('hidden layers', 0, 16, 1),
    'hidden units': hp.quniform('hidden units', 100, 1000, 1),
    'dropout rate': hp.uniform('dropout rate', .01, .1),
    'learning rate': hp.uniform('learning rate', 1e-07, 1e-05 )}

constant_parameters = {
    'shape': 'square',
    'optimizer': 'RMSprop',
    'kernel initializer': 'lecun_normal',
    'patience': 30
}

trials = Trials()
best = fmin(f_nn, space, algo=tpe.suggest, max_evals=100, trials=trials)

print('best: ')
print(best)



{'batch size': 87.0, 'dropout rate': 0.09512000755425924, 'hidden layers': 9.0, 'hidden units': 515.0, 'learning rate': 6.965558106521671e-06, 'shape': 'square', 'optimizer': 'RMSprop', 'kernel initializer': 'lecun_normal', 'patience': 30}
Class weights:
[ 0.56575067  4.30224261]
Train on 8441 samples, validate on 610 samples
Epoch 1/1000
0s - loss: 0.4853 - binary_accuracy: 0.7898 - val_loss: 1.2524 - val_binary_accuracy: 0.8803
Epoch 2/1000
0s - loss: 0.3685 - binary_accuracy: 0.8723 - val_loss: 0.8634 - val_binary_accuracy: 0.8803
Epoch 3/1000
0s - loss: 0.3334 - binary_accuracy: 0.8754 - val_loss: 0.6658 - val_binary_accuracy: 0.8934
Epoch 4/1000
0s - loss: 0.3187 - binary_accuracy: 0.8756 - val_loss: 0.6001 - val_binary_accuracy: 0.8934
Epoch 5/1000
0s - loss: 0.3050 - binary_accuracy: 0.8833 - val_loss: 0.6089 - val_binary_accuracy: 0.8951
Epoch 6/1000
0s - loss: 0.2982 - binary_accuracy: 0.8860 - val_loss: 0.5856 - val_binary_accuracy: 0.8967
Epoch 7/1000
0s - loss: 0.2898 - b

0s - loss: 0.1977 - binary_accuracy: 0.9209 - val_loss: 0.3259 - val_binary_accuracy: 0.8902
Epoch 76/1000
0s - loss: 0.1975 - binary_accuracy: 0.9234 - val_loss: 0.3024 - val_binary_accuracy: 0.9016
Epoch 77/1000
0s - loss: 0.2005 - binary_accuracy: 0.9220 - val_loss: 0.3096 - val_binary_accuracy: 0.9049
Epoch 78/1000
0s - loss: 0.2018 - binary_accuracy: 0.9213 - val_loss: 0.2956 - val_binary_accuracy: 0.8967
Epoch 79/1000
0s - loss: 0.1957 - binary_accuracy: 0.9256 - val_loss: 0.3014 - val_binary_accuracy: 0.9000
Epoch 80/1000
0s - loss: 0.1990 - binary_accuracy: 0.9234 - val_loss: 0.3030 - val_binary_accuracy: 0.9033
Epoch 81/1000
0s - loss: 0.1987 - binary_accuracy: 0.9249 - val_loss: 0.3053 - val_binary_accuracy: 0.9049
Epoch 82/1000
0s - loss: 0.1980 - binary_accuracy: 0.9228 - val_loss: 0.3067 - val_binary_accuracy: 0.9049
Epoch 83/1000
0s - loss: 0.1932 - binary_accuracy: 0.9252 - val_loss: 0.3010 - val_binary_accuracy: 0.9033
Epoch 84/1000
0s - loss: 0.1968 - binary_accuracy: 

1s - loss: 0.2509 - binary_accuracy: 0.9061 - val_loss: 0.5110 - val_binary_accuracy: 0.8902
Epoch 31/1000
1s - loss: 0.2495 - binary_accuracy: 0.9045 - val_loss: 0.5053 - val_binary_accuracy: 0.8885
Epoch 32/1000
1s - loss: 0.2474 - binary_accuracy: 0.9063 - val_loss: 0.4923 - val_binary_accuracy: 0.8885
Epoch 33/1000
1s - loss: 0.2478 - binary_accuracy: 0.9059 - val_loss: 0.4762 - val_binary_accuracy: 0.8869
Epoch 34/1000
1s - loss: 0.2451 - binary_accuracy: 0.9069 - val_loss: 0.4800 - val_binary_accuracy: 0.8885
Epoch 35/1000
1s - loss: 0.2480 - binary_accuracy: 0.9038 - val_loss: 0.4701 - val_binary_accuracy: 0.8852
Epoch 36/1000
1s - loss: 0.2406 - binary_accuracy: 0.9089 - val_loss: 0.4732 - val_binary_accuracy: 0.8836
Epoch 37/1000
1s - loss: 0.2458 - binary_accuracy: 0.9058 - val_loss: 0.4499 - val_binary_accuracy: 0.8869
Epoch 38/1000
1s - loss: 0.2426 - binary_accuracy: 0.9056 - val_loss: 0.4511 - val_binary_accuracy: 0.8820
Epoch 39/1000
1s - loss: 0.2379 - binary_accuracy: 

1s - loss: 0.2045 - binary_accuracy: 0.9242 - val_loss: 0.3371 - val_binary_accuracy: 0.9016
Epoch 108/1000
1s - loss: 0.2036 - binary_accuracy: 0.9217 - val_loss: 0.3442 - val_binary_accuracy: 0.9033
Epoch 109/1000
1s - loss: 0.2004 - binary_accuracy: 0.9234 - val_loss: 0.3543 - val_binary_accuracy: 0.9033
Epoch 110/1000
1s - loss: 0.2042 - binary_accuracy: 0.9217 - val_loss: 0.3453 - val_binary_accuracy: 0.9049
Epoch 111/1000
1s - loss: 0.2027 - binary_accuracy: 0.9241 - val_loss: 0.3362 - val_binary_accuracy: 0.9033
Epoch 112/1000
1s - loss: 0.1992 - binary_accuracy: 0.9223 - val_loss: 0.3313 - val_binary_accuracy: 0.8984
Epoch 113/1000
1s - loss: 0.2001 - binary_accuracy: 0.9241 - val_loss: 0.3263 - val_binary_accuracy: 0.9049
Epoch 114/1000
1s - loss: 0.2032 - binary_accuracy: 0.9206 - val_loss: 0.3345 - val_binary_accuracy: 0.9033
Epoch 115/1000
1s - loss: 0.2014 - binary_accuracy: 0.9211 - val_loss: 0.3523 - val_binary_accuracy: 0.9049
Epoch 116/1000
1s - loss: 0.2020 - binary_a

KeyboardInterrupt: 

In [ ]:
space = {
    'batch size' : hp.quniform('batch size', 10, 100, 1),
    'hidden layers': hp.quniform('hidden layers', 0, 16, 1),
    'hidden units': hp.quniform('hidden units', 100, 1000, 1),
    'dropout rate': hp.uniform('dropout rate', .01, .1),
    'learning rate': hp.uniform('learning rate', 1e-07, 1e-05 )}

constant_parameters = {
    'shape': 'square',
    'optimizer': 'Adadelta',
    'kernel initializer': 'lecun_normal',
    'patience': 30
}

trials = Trials()
best = fmin(f_nn, space, algo=tpe.suggest, max_evals=100, trials=trials)

print('best: ')
print(best)



{'batch size': 10.0, 'dropout rate': 0.031521600951354235, 'hidden layers': 12.0, 'hidden units': 729.0, 'learning rate': 8.25134476482145e-06, 'shape': 'square', 'optimizer': 'Adadelta', 'kernel initializer': 'lecun_normal', 'patience': 30}
Class weights:
[ 0.56575067  4.30224261]
Train on 8441 samples, validate on 610 samples
Epoch 1/1000
7s - loss: 0.3177 - binary_accuracy: 0.8792 - val_loss: 0.3201 - val_binary_accuracy: 0.8918
Epoch 2/1000
6s - loss: 0.2672 - binary_accuracy: 0.8961 - val_loss: 0.3330 - val_binary_accuracy: 0.8902
Epoch 3/1000
6s - loss: 0.2562 - binary_accuracy: 0.9000 - val_loss: 0.2611 - val_binary_accuracy: 0.8902
Epoch 4/1000
6s - loss: 0.2425 - binary_accuracy: 0.9057 - val_loss: 0.3429 - val_binary_accuracy: 0.8459
Epoch 5/1000
6s - loss: 0.2409 - binary_accuracy: 0.9075 - val_loss: 0.2807 - val_binary_accuracy: 0.8951
Epoch 6/1000
6s - loss: 0.2352 - binary_accuracy: 0.9077 - val_loss: 0.2830 - val_binary_accuracy: 0.8902
Epoch 7/1000
6s - loss: 0.2306 -

0s - loss: 0.1964 - binary_accuracy: 0.9197 - val_loss: 0.3072 - val_binary_accuracy: 0.8721
Epoch 38/1000
0s - loss: 0.1977 - binary_accuracy: 0.9229 - val_loss: 0.2803 - val_binary_accuracy: 0.8984
Epoch 39/1000
0s - loss: 0.1958 - binary_accuracy: 0.9222 - val_loss: 0.2808 - val_binary_accuracy: 0.8836
Epoch 40/1000
0s - loss: 0.1954 - binary_accuracy: 0.9224 - val_loss: 0.2954 - val_binary_accuracy: 0.8787
Epoch 41/1000
0s - loss: 0.1962 - binary_accuracy: 0.9228 - val_loss: 0.2881 - val_binary_accuracy: 0.8885
Epoch 42/1000
0s - loss: 0.1957 - binary_accuracy: 0.9245 - val_loss: 0.2852 - val_binary_accuracy: 0.8852
 32/610 [>.............................] - ETA: 0s         NR.AhR score on test: 0.89174


{'batch size': 84.0, 'dropout rate': 0.05396371921151314, 'hidden layers': 12.0, 'hidden units': 104.0, 'learning rate': 7.77978018617794e-06, 'shape': 'square', 'optimizer': 'Adadelta', 'kernel initializer': 'lecun_normal', 'patience': 30}
Class weights:
[ 0.56575067  4.30224261]

0s - loss: 0.2198 - binary_accuracy: 0.9155 - val_loss: 0.2579 - val_binary_accuracy: 0.8967
Epoch 69/1000
0s - loss: 0.2218 - binary_accuracy: 0.9153 - val_loss: 0.2598 - val_binary_accuracy: 0.8918
Epoch 70/1000
0s - loss: 0.2196 - binary_accuracy: 0.9148 - val_loss: 0.2590 - val_binary_accuracy: 0.8967
Epoch 71/1000
0s - loss: 0.2203 - binary_accuracy: 0.9123 - val_loss: 0.2583 - val_binary_accuracy: 0.8934
Epoch 72/1000
0s - loss: 0.2185 - binary_accuracy: 0.9168 - val_loss: 0.2611 - val_binary_accuracy: 0.8951
Epoch 73/1000
0s - loss: 0.2160 - binary_accuracy: 0.9186 - val_loss: 0.2608 - val_binary_accuracy: 0.8951
Epoch 74/1000
0s - loss: 0.2231 - binary_accuracy: 0.9135 - val_loss: 0.2587 - val_binary_accuracy: 0.9000
Epoch 75/1000
1s - loss: 0.2166 - binary_accuracy: 0.9173 - val_loss: 0.2589 - val_binary_accuracy: 0.8951
Epoch 76/1000
1s - loss: 0.2174 - binary_accuracy: 0.9149 - val_loss: 0.2606 - val_binary_accuracy: 0.8951
Epoch 77/1000
1s - loss: 0.2158 - binary_accuracy: 

1s - loss: 0.2088 - binary_accuracy: 0.9202 - val_loss: 0.4029 - val_binary_accuracy: 0.8934
Epoch 44/1000
1s - loss: 0.2094 - binary_accuracy: 0.9166 - val_loss: 0.3934 - val_binary_accuracy: 0.8967
Epoch 45/1000
1s - loss: 0.2058 - binary_accuracy: 0.9196 - val_loss: 0.3756 - val_binary_accuracy: 0.8508
Epoch 46/1000
1s - loss: 0.2053 - binary_accuracy: 0.9200 - val_loss: 0.3824 - val_binary_accuracy: 0.8574
Epoch 47/1000
1s - loss: 0.2057 - binary_accuracy: 0.9205 - val_loss: 0.3908 - val_binary_accuracy: 0.8934
Epoch 48/1000
1s - loss: 0.2044 - binary_accuracy: 0.9202 - val_loss: 0.3918 - val_binary_accuracy: 0.8885
Epoch 49/1000
1s - loss: 0.2020 - binary_accuracy: 0.9186 - val_loss: 0.3839 - val_binary_accuracy: 0.8918
Epoch 50/1000
1s - loss: 0.2011 - binary_accuracy: 0.9220 - val_loss: 0.3621 - val_binary_accuracy: 0.8934
Epoch 51/1000
1s - loss: 0.1971 - binary_accuracy: 0.9245 - val_loss: 0.3783 - val_binary_accuracy: 0.8951
Epoch 52/1000
1s - loss: 0.2003 - binary_accuracy: 

3s - loss: 0.2508 - binary_accuracy: 0.9017 - val_loss: 0.3093 - val_binary_accuracy: 0.8967
Epoch 7/1000
3s - loss: 0.2415 - binary_accuracy: 0.9061 - val_loss: 0.2810 - val_binary_accuracy: 0.8885
Epoch 8/1000
3s - loss: 0.2409 - binary_accuracy: 0.9029 - val_loss: 0.2965 - val_binary_accuracy: 0.8656
Epoch 9/1000
3s - loss: 0.2385 - binary_accuracy: 0.9058 - val_loss: 0.2787 - val_binary_accuracy: 0.8869
Epoch 10/1000
3s - loss: 0.2288 - binary_accuracy: 0.9136 - val_loss: 0.2792 - val_binary_accuracy: 0.8820
Epoch 11/1000
3s - loss: 0.2312 - binary_accuracy: 0.9088 - val_loss: 0.2649 - val_binary_accuracy: 0.8869
Epoch 12/1000
3s - loss: 0.2255 - binary_accuracy: 0.9128 - val_loss: 0.2941 - val_binary_accuracy: 0.8738
Epoch 13/1000
3s - loss: 0.2224 - binary_accuracy: 0.9135 - val_loss: 0.2894 - val_binary_accuracy: 0.9033
Epoch 14/1000
3s - loss: 0.2238 - binary_accuracy: 0.9116 - val_loss: 0.2938 - val_binary_accuracy: 0.8885
Epoch 15/1000
3s - loss: 0.2191 - binary_accuracy: 0.9

In [14]:
from hyperopt import space_eval 

for target in ['NR.AhR', 'NR.AR', 'NR.AR.LBD', 'NR.Aromatase', 'NR.ER', 'NR.ER.LBD', 
               'NR.PPAR.gamma', 'SR.ARE', 'SR.ATAD5', 'SR.HSE', 'SR.MMP', 'SR.p53']:
    
    x_train, y_train, x_test, y_test = data(target)

    space = {
        'batch size' : hp.quniform('batch size', 10, 100, 1),
        'hidden layers': hp.quniform('hidden layers', 0, 16, 1),
        'hidden units': hp.quniform('hidden units', 100, 1000, 1),
        'dropout rate': hp.uniform('dropout rate', .01, .1),
        'learning rate': hp.uniform('learning rate', 1e-07, 1e-05 )}

    constant_parameters = {
        'shape': 'square',
        'optimizer': 'adam',
        'kernel initializer': 'lecun_normal',
        'patience': 30
    }

    trials = Trials()
    best = fmin(f_nn, space, algo=tpe.suggest, max_evals=25, trials=trials)

    print('======== best for ' + target + ': ')
    print(space_eval(space, best))



{'batch size': 22.0, 'dropout rate': 0.07136936311571818, 'hidden layers': 7.0, 'hidden units': 268.0, 'learning rate': 9.242742375853505e-06, 'shape': 'square', 'optimizer': 'adam', 'kernel initializer': 'lecun_normal', 'patience': 30}
Class weights:
[ 0.56575067  4.30224261]
 32/610 [>.............................] - ETA: 0s         NR.AhR score on test: 0.89799


{'batch size': 82.0, 'dropout rate': 0.014077402974367395, 'hidden layers': 2.0, 'hidden units': 392.0, 'learning rate': 6.67985064253016e-06, 'shape': 'square', 'optimizer': 'adam', 'kernel initializer': 'lecun_normal', 'patience': 30}
Class weights:
[ 0.56575067  4.30224261]
 32/610 [>.............................] - ETA: 0s         NR.AhR score on test: 0.89161


{'batch size': 42.0, 'dropout rate': 0.09251178661959933, 'hidden layers': 7.0, 'hidden units': 909.0, 'learning rate': 5.355445692499389e-06, 'shape': 'square', 'optimizer': 'adam', 'kernel initializer': 'lecun_normal', 'patience': 30}
Class weights:
[ 0.5657

 32/610 [>.............................] - ETA: 0s         NR.AhR score on test: 0.89033


{'batch size': 25.0, 'dropout rate': 0.03106785170629097, 'hidden layers': 13.0, 'hidden units': 461.0, 'learning rate': 6.338565998807057e-07, 'shape': 'square', 'optimizer': 'adam', 'kernel initializer': 'lecun_normal', 'patience': 30}
Class weights:
[ 0.56575067  4.30224261]
 32/610 [>.............................] - ETA: 1s         NR.AhR score on test: 0.86016


{'batch size': 29.0, 'dropout rate': 0.06503501904889716, 'hidden layers': 9.0, 'hidden units': 594.0, 'learning rate': 4.928405421328021e-06, 'shape': 'square', 'optimizer': 'adam', 'kernel initializer': 'lecun_normal', 'patience': 30}
Class weights:
[ 0.56575067  4.30224261]
 32/610 [>.............................] - ETA: 1s         NR.AhR score on test: 0.86511
======== best for NR.AhR: 
{'batch size': 27.0, 'dropout rate': 0.037447677802347416, 'hidden layers': 10.0, 'hidden units': 407.0, 'learning rate': 4.27275024188765e-06}



Class weights:
[  0.52065581  12.60313316]
 32/586 [>.............................] - ETA: 1s          NR.AR score on test: 0.76800


{'batch size': 12.0, 'dropout rate': 0.0322678967833666, 'hidden layers': 16.0, 'hidden units': 103.0, 'learning rate': 6.506330961847355e-06, 'shape': 'square', 'optimizer': 'adam', 'kernel initializer': 'lecun_normal', 'patience': 30}
Class weights:
[  0.52065581  12.60313316]
 32/586 [>.............................] - ETA: 1s          NR.AR score on test: 0.80328


{'batch size': 11.0, 'dropout rate': 0.038616255221048224, 'hidden layers': 14.0, 'hidden units': 207.0, 'learning rate': 6.428815730482679e-06, 'shape': 'square', 'optimizer': 'adam', 'kernel initializer': 'lecun_normal', 'patience': 30}
Class weights:
[  0.52065581  12.60313316]
 32/586 [>.............................] - ETA: 1s          NR.AR score on test: 0.75929


{'batch size': 19.0, 'dropout rate': 0.035114738391035724, 'hidden layers': 14.0, 'hidden units': 109.0, 'learning rate': 

Class weights:
[  0.51796372  14.41693811]
 32/582 [>.............................] - ETA: 1s      NR.AR.LBD score on test: 0.49260


{'batch size': 39.0, 'dropout rate': 0.028099177036559966, 'hidden layers': 16.0, 'hidden units': 303.0, 'learning rate': 6.997936744715498e-06, 'shape': 'square', 'optimizer': 'adam', 'kernel initializer': 'lecun_normal', 'patience': 30}
Class weights:
[  0.51796372  14.41693811]
 32/582 [>.............................] - ETA: 1s      NR.AR.LBD score on test: 0.55684


{'batch size': 93.0, 'dropout rate': 0.023109411761090885, 'hidden layers': 2.0, 'hidden units': 757.0, 'learning rate': 1.994323226570051e-06, 'shape': 'square', 'optimizer': 'adam', 'kernel initializer': 'lecun_normal', 'patience': 30}
Class weights:
[  0.51796372  14.41693811]
 32/582 [>.............................] - ETA: 0s      NR.AR.LBD score on test: 0.76307


{'batch size': 75.0, 'dropout rate': 0.013374427869563022, 'hidden layers': 13.0, 'hidden units': 458.0, 'learning rate':

Class weights:
[ 0.52676296  9.84126984]
 32/528 [>.............................] - ETA: 1s   NR.Aromatase score on test: 0.69965


{'batch size': 36.0, 'dropout rate': 0.06552400634466429, 'hidden layers': 0.0, 'hidden units': 648.0, 'learning rate': 1.7867758860417195e-07, 'shape': 'square', 'optimizer': 'adam', 'kernel initializer': 'lecun_normal', 'patience': 30}
Class weights:
[ 0.52676296  9.84126984]
 32/528 [>.............................] - ETA: 0s   NR.Aromatase score on test: 0.79073


{'batch size': 80.0, 'dropout rate': 0.022171283210297765, 'hidden layers': 16.0, 'hidden units': 263.0, 'learning rate': 1.953967863667563e-06, 'shape': 'square', 'optimizer': 'adam', 'kernel initializer': 'lecun_normal', 'patience': 30}
Class weights:
[ 0.52676296  9.84126984]
 32/528 [>.............................] - ETA: 1s   NR.Aromatase score on test: 0.73127


{'batch size': 75.0, 'dropout rate': 0.09005642766930387, 'hidden layers': 2.0, 'hidden units': 286.0, 'learning rate': 3.75073

...

 32/555 [>.............................] - ETA: 0s         SR.ARE score on test: 0.79507
======== best for SR.ARE: 
{'batch size': 42.0, 'dropout rate': 0.07035786455532222, 'hidden layers': 1.0, 'hidden units': 197.0, 'learning rate': 9.392678443347974e-06}


{'batch size': 26.0, 'dropout rate': 0.017059349876564302, 'hidden layers': 1.0, 'hidden units': 474.0, 'learning rate': 5.900253675506423e-06, 'shape': 'square', 'optimizer': 'adam', 'kernel initializer': 'lecun_normal', 'patience': 30}
Class weights:
[  0.52016667  12.89669421]
 32/622 [>.............................] - ETA: 0s       SR.ATAD5 score on test: 0.78821


{'batch size': 95.0, 'dropout rate': 0.07355255275435046, 'hidden layers': 2.0, 'hidden units': 852.0, 'learning rate': 3.6122381597964968e-06, 'shape': 'square', 'optimizer': 'adam', 'kernel initializer': 'lecun_normal', 'patience': 30}
Class weights:
[  0.52016667  12.89669421]
 32/622 [>.............................] - ETA: 0s       SR.ATAD5 score on test: 0.707

Class weights:
[  0.52016667  12.89669421]
 32/622 [>.............................] - ETA: 1s       SR.ATAD5 score on test: 0.68696


{'batch size': 38.0, 'dropout rate': 0.09977000941695163, 'hidden layers': 9.0, 'hidden units': 824.0, 'learning rate': 7.066825515044912e-06, 'shape': 'square', 'optimizer': 'adam', 'kernel initializer': 'lecun_normal', 'patience': 30}
Class weights:
[  0.52016667  12.89669421]
 32/622 [>.............................] - ETA: 0s       SR.ATAD5 score on test: 0.74288


{'batch size': 18.0, 'dropout rate': 0.054055382403718655, 'hidden layers': 14.0, 'hidden units': 374.0, 'learning rate': 4.312985518707168e-06, 'shape': 'square', 'optimizer': 'adam', 'kernel initializer': 'lecun_normal', 'patience': 30}
Class weights:
[  0.52016667  12.89669421]
 32/622 [>.............................] - ETA: 1s       SR.ATAD5 score on test: 0.82678


{'batch size': 63.0, 'dropout rate': 0.05174275428841806, 'hidden layers': 14.0, 'hidden units': 377.0, 'learning rate': 4

Class weights:
[ 0.52744705  9.60844749]
 32/610 [>.............................] - ETA: 1s         SR.HSE score on test: 0.79105


{'batch size': 62.0, 'dropout rate': 0.08048115860238501, 'hidden layers': 5.0, 'hidden units': 956.0, 'learning rate': 6.328540612978295e-06, 'shape': 'square', 'optimizer': 'adam', 'kernel initializer': 'lecun_normal', 'patience': 30}
Class weights:
[ 0.52744705  9.60844749]
 32/610 [>.............................] - ETA: 0s         SR.HSE score on test: 0.82692


{'batch size': 78.0, 'dropout rate': 0.042249533265904415, 'hidden layers': 1.0, 'hidden units': 607.0, 'learning rate': 6.924519848071295e-06, 'shape': 'square', 'optimizer': 'adam', 'kernel initializer': 'lecun_normal', 'patience': 30}
Class weights:
[ 0.52744705  9.60844749]
 32/610 [>.............................] - ETA: 0s         SR.HSE score on test: 0.85080


{'batch size': 78.0, 'dropout rate': 0.040421274398621414, 'hidden layers': 1.0, 'hidden units': 611.0, 'learning rate': 6.909218

Class weights:
[ 0.59250549  3.20254237]
 32/543 [>.............................] - ETA: 1s         SR.MMP score on test: 0.92039


{'batch size': 46.0, 'dropout rate': 0.075362197253205, 'hidden layers': 15.0, 'hidden units': 516.0, 'learning rate': 1.2218986007862182e-06, 'shape': 'square', 'optimizer': 'adam', 'kernel initializer': 'lecun_normal', 'patience': 30}
Class weights:
[ 0.59250549  3.20254237]
 32/543 [>.............................] - ETA: 1s         SR.MMP score on test: 0.67333


{'batch size': 58.0, 'dropout rate': 0.053522242402782964, 'hidden layers': 14.0, 'hidden units': 746.0, 'learning rate': 9.535339237161361e-06, 'shape': 'square', 'optimizer': 'adam', 'kernel initializer': 'lecun_normal', 'patience': 30}
Class weights:
[ 0.59250549  3.20254237]
 32/543 [>.............................] - ETA: 1s         SR.MMP score on test: 0.90769


{'batch size': 91.0, 'dropout rate': 0.03918473049609136, 'hidden layers': 13.0, 'hidden units': 605.0, 'learning rate': 2.22818

 32/616 [>.............................] - ETA: 0s         SR.p53 score on test: 0.77888


{'batch size': 95.0, 'dropout rate': 0.029162052921794843, 'hidden layers': 10.0, 'hidden units': 518.0, 'learning rate': 7.255870737968997e-06, 'shape': 'square', 'optimizer': 'adam', 'kernel initializer': 'lecun_normal', 'patience': 30}
Class weights:
[ 0.53388103  7.87876106]
 32/616 [>.............................] - ETA: 1s         SR.p53 score on test: 0.73650


{'batch size': 62.0, 'dropout rate': 0.09163087648880294, 'hidden layers': 7.0, 'hidden units': 164.0, 'learning rate': 6.891298693882184e-06, 'shape': 'square', 'optimizer': 'adam', 'kernel initializer': 'lecun_normal', 'patience': 30}
Class weights:
[ 0.53388103  7.87876106]
 32/616 [>.............................] - ETA: 0s         SR.p53 score on test: 0.75169


{'batch size': 92.0, 'dropout rate': 0.09929882808843414, 'hidden layers': 0.0, 'hidden units': 165.0, 'learning rate': 8.976590425443003e-06, 'shape': 'square', 'optimi